In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()
API_KEY = '7781c628db944d3dbd8185950252402'
BASE_URL = "http://api.weatherapi.com/v1/history.json"

def get_last_24h_weather(city: str):
    """
    Fetch the last 24 hours of hourly weather data for a given city.

    Args:
        city (str): City name (e.g., "London").

    Returns:
        pd.DataFrame: Weather data for the last 24 hours.
    """
    # Get yesterday's date (since history API doesn't support live past 24h)
    yesterday = (datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%d')
    
    params = {
        "key": API_KEY,
        "q": city,
        "dt": yesterday,
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        hourly_data = data["forecast"]["forecastday"][0]["hour"]
        
        # Convert to DataFrame
        df = pd.DataFrame(hourly_data)
        df["time"] = pd.to_datetime(df["time"])
        
        # Select relevant columns
        df = df[["time", "temp_c", "humidity", "condition"]]
        df["weather_description"] = df["condition"].apply(lambda x: x["text"])
        df.drop(columns=["condition"], inplace=True)

        return df

    else:
        print(f"Error fetching data: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Get weather for London
df_weather = get_last_24h_weather("London")

# Display DataFrame

print(df_weather)

Error fetching data: 401 - {"error":{"code":1002,"message":"API key is invalid or not provided."}}
Empty DataFrame
Columns: []
Index: []


/var/folders/k5/0ckgxv2x46n7_p0qwn2h0hhh0000gn/T/ipykernel_58865/1887099356.py:23: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  yesterday = (datetime.utcnow() - timedelta(days=1)).strftime('%Y-%m-%d')


In [12]:
load_dotenv()
API_KEY = os.getenv("WEATHERAPI_KEY")  # Ensure the variable name matches the .env file


In [13]:
print(f"Loaded API Key: {API_KEY}")


Loaded API Key: None
